# 1.关于对话模型中消息（message）的使用

标准的对话模型的调用过程：

invoke() 的输入可以是多种类型，经典类型有：

- 字符串类型
- 消息列表

invoke() 的输出类型：

- BaseModel的子类：AIMessage


1. 输入类型（Input Types）

      **invoke()** 具有极强的兼容性，它会根据你传入的类型自动处理数据：

    - 字符串类型 (str)

    表现：最简单的调用方式。

    底层：LangChain 会自动将其包装成一个包含单条消息的列表：[HumanMessage(content="你的内容")]。

    适用场景：单次、简单的问答。

    - 消息列表 (List[BaseMessage])

        - 表现：传入一个包含 SystemMessage、HumanMessage、AIMessage 的列表。
        - 
        - 底层：直接将完整的对话上下文发送给模型。
        - 
        - 适用场景：多轮对话、设定复杂人设、提供背景资料。这是构建 Agent 的标准做法。

    - 字典类型 (dict)
      - 适用场景：当你使用 PromptTemplate（提示词模板）时。

      - 示例：chain.invoke({"topic": "猫", "style": "幽默"})。


2. 输出类型（Output Types）
    对于对话模型（Chat Models），invoke() 的返回结果永远是一个 AIMessage 对象。

    - 核心属性：
      - .content：模型生成的纯文本回复。-
      - .response_metadata：包含模型名、Token 消耗统计、结束原因（finish_reason）等原始数据的字典。-
      - .id：该消息的唯一标识符。-
      - .tool_calls：如果模型决定调用某个工具（如搜索、绘图），这里会包含工具的相关参数。

## LangChain有一些内置的消息类型：

- 🔥 **SystemMessage** ：设定AI行为规则或背景信息。比如设定AI的初始状态、行为模式或对话的总体目标。比如“作为一个代码专家”，或者“返回json格式”。通常作为输入消息序列中的第一个传递。
- 🔥 **HumanMessage** ：表示来自用户输入。比如“实现 一个快速排序方法”
- 🔥 **AIMessage** ：存储AI回复的内容。这可以是文本，也可以是调用工具的请求
- 🔥 **ChatMessage** ：可以自定义角色的通用消息类型
- 🔥 **FunctionMessage/ToolMessage** ：函数调用/工具消息，用于函数调用结果的消息类型


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import os
import dotenv

# 前提：加载配置文件
dotenv.load_dotenv()
os.environ["SILICON_FLOW_API_KEY"] = os.getenv("SILICON_FLOW_API_KEY")
os.environ["SILICON_FLOW_BASE_URL"] = os.getenv("SILICON_FLOW_BASE_URL")

# 1、获取对话模型
Chat_model = ChatOpenAI(
    api_key=os.getenv("SILICON_FLOW_API_KEY"),
    base_url=os.getenv("SILICON_FLOW_BASE_URL"),
    model="Qwen/Qwen3-8B",
)

# 2、调用
# 此时你可以自由使用 LangChain 的消息对象
messages = [
    SystemMessage(content="你是一位乐于助人的智能小助手"),
    HumanMessage(content="你好，请你介绍一下你自己"),
]
response = Chat_model.invoke(messages)

# 3、处理响应数据
print(response)

content='你好呀！我是一个乐于助人的智能助手，名字叫Qwen，是通义实验室研发的超大规模语言模型。我的目标是能够帮助用户解答各种问题、提供信息、创作文字，甚至进行一些简单的推理和逻辑分析。我可以和你聊天、讲故事、写作文、做数学题，还能提供生活小贴士和学习建议哦！你有什么想知道的或者需要帮忙的吗？随时告诉我，我会尽力帮你！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 31, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '019c1e51eeed5976c4cf8ba2b00a8025', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c1e51-ee1d-78c3-afcf-9f8cd5b18635-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 31, 'output_tokens': 95, 'total_tokens': 126, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}}


In [ ]:
# 1.导入相关包
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ChatMessage

system_message = SystemMessage(content="你是一个专业的数据科学家")
human_message = HumanMessage(content="解释一下随机森林算法")
ai_message = AIMessage(content="随机森林是一种集成学习方法...")
custom_message = ChatMessage(
    role="analyst",
    content="补充一点关于超参数调优的信息",
)
print(system_message.content)
print(system_message.content)
print(human_message.content)
print(ai_message.content)
print(custom_message.content)

你是一个专业的数据科学家
{}
解释一下随机森林算法
随机森林是一种集成学习方法...
补充一点关于超参数调优的信息


## 对话模型示例代码

response 属性详解
1. .content (最常用的部分)
    - 类型：字符串 (str)。
    - 内容：这是 AI 真正回复给你的文本主体。
    - 用途：在你的项目中，这里存放的就是 AI 写出来的小说正文或推文标题。

2. .response_metadata (统计与元数据)
    - 这是一个字典 (dict)，包含了本次请求的后台数据。不同的模型供应商（如硅基流动、OpenAI）提供的字段略有   - 不同，但通常包含：
    - token_usage：极其重要！包含 prompt_tokens（输入）、completion_tokens（输出）和 total_tokens。
    - model_name：本次实际运行的模型 ID。
    - finish_reason：模型停止的原因。例如 stop 代表正常结束，length 代表因为达到 max_tokens 限制而被截    - 断了。

3. .id (消息唯一标识)
    - 用途：LangChain 为每条消息生成的唯一 ID，在需要追踪特定对话历史或存入数据库时非常有用。

4. .additional_kwargs (扩展参数)
    - 内容：包含模型供应商特有的一些参数。例如，某些模型会返回 logprobs（每个字生成的概率）或原始的 API 响应。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv(
    # 在工程根目录只保留一个 .env，所有模块都引用它。这是最不容易出错的方式。
    override=True,  # 如果加载.env文件出现了同名环境变脸，会进行覆盖
    # dotenv_path=".env",  # 可以只当需要加载的路径
)

"""
    # 1.LangChain 的底层代码默认会寻找名为 "OPENAI_API_KEY" 和 "OPENAI_API_BASE" 的变量
    #   这里通过 os.environ 将你自定义的变量名（如 OPENAI_API_KEY1）映射到标准变量名上
    os.environ["SILICON_FLOW_API_KEY"] = os.getenv("SILICON_FLOW_API_KEY")
    os.environ["SILICON_FLOW_BASE_URL"] = os.getenv("SILICON_FLOW_BASE_URL")

    # 因此将变量映射好后，初始化模型时，就可以不用指定API参数了
    # 2. 初始化模型：此时 LangChain 会自动找到上面的环境变量，无需传参
    chat_model = ChatOpenAI(
        model="Qwen/Qwen2.5-7B-Instruct",
        temperature=0.7
    )
"""

# 1. 影响范围：仅限当前进程（及其子进程）
#     当前文件：当你运行这个 .py 文件时，它会创建一个操作系统进程。这两行代码在内存中修改了该进程的环境变量字典。
#
#     整个工程：如果你的工程中有其他 .py 文件，只要它们不是由当前这个文件启动的（比如你单独开了另一个终端运行另一个文件），
#     那么它们无法读取到这里的映射。
#
#     子进程：如果你在这个脚本中调用了其他子程序（比如通过 os.system 运行脚本），那么这些子程序通常会继承这个映射。
#
# 2. 生命周期：随运行结束而消失
#     内存存储：os.environ 的修改是存储在内存中的。
#
#     持久性：它不会修改你硬盘上的 .env 文件，也不会修改操作系统的全局设置。一旦你点击 VS Code 的停止按钮或者程序运行结束，
#     这个映射就会立即被销毁。下次运行，必须重新执行这两行代码才能生效。


# 初始化模型
# 提示：硅基流动平台模型通常建议开启 streaming=True 来获得更好的交互体验
chat_model = ChatOpenAI(
    api_key=os.getenv("SILICON_FLOW_API_KEY"),
    base_url=os.getenv("SILICON_FLOW_BASE_URL"),
    model="Qwen/Qwen2.5-7B-Instruct",  # 确保模型名称准确
    temperature=0.7,
)

# --- 核心：理解消息列表 (Messages List) ---
# 消息列表是一个“历史记录堆栈”，模型通过阅读这个列表来获取上下文
messages = [
    # 1. SystemMessage (系统消息)
    # 作用：设定“人设”和“规则”。它是 AI 的底层逻辑，不会轻易改变。
    SystemMessage(
        content="你是一位精通小红书爆款文案的创作助手，说话幽默，喜欢用 Emoji。"
    ),
    # 2. HumanMessage (用户消息)
    # 作用：模拟你对 AI 说的话。
    HumanMessage(content="你好，我想写一个关于‘AI写小说’的推文标题。"),
    # 3. AIMessage (助手消息 - 模拟记忆)
    # 作用：代表 AI 之前说的话。通过手动添加 AIMessage，你可以“伪造”记忆，
    # 从而让模型在接下来的回复中保持逻辑一致。
    AIMessage(
        content="没问题！我们可以从‘副业赚钱’或者‘科技黑客’两个角度切入。你更倾向哪个？"
    ),
    # 4. 再次提问（模拟多轮对话）
    HumanMessage(content="选科技黑客角度，要酷一点。"),
]

# 调用模型
# 模型会阅读上面所有的消息，意识到现在是对话的“第四回合”
response = chat_model.invoke(messages)

# --- 处理响应数据 ---
print("--- 原始响应对象 ---")
print(response)

print("\n--- 纯文本回复 ---")
print(response.content)

print("\n--- Token 消耗详情 ---")
# 记录成本的关键数据
usage = response.response_metadata.get("token_usage", {})
print(response.response_metadata)
print(usage)
print(f"输入: {usage.get('prompt_tokens')} | 输出: {usage.get('completion_tokens')}")

# ====================================================================== #
# 1. 初始化消息列表，设定系统人设
# 这里的 history[0] 永远是你的“小说家”身份
history = [SystemMessage(content="你是一位精通情节设计的玄幻小说家。")]

print("--- 🤖 小说创作助手已就绪 (输入 'exit' 或 '退出' 结束对话) ---")

while True:
    # 获取用户输入
    user_input = input("\n👤 你的指令: ")

    # --- 核心改进：添加退出逻辑 ---
    # .strip() 去除空格，.lower() 统一转为小写，增加容错率
    if user_input.strip().lower() in ["exit", "quit", "退出", "stop"]:
        print("🚀 正在保存会话并退出... 创作辛苦了！")
        break

    # 将有效的用户输入存入历史
    history.append(HumanMessage(content=user_input))

    # --- 核心改进：人设保护机制 ---
    # 如果直接使用 history[-10:]，当对话很长时，history[0] (SystemMessage) 会被丢弃
    # 导致 AI 忘记自己是“小说家”。
    # 优化方案：始终保留 history[0]，再加上最近的 10 条对话
    if len(history) > 10:
        # 拼接：[系统消息] + [最近的10条消息]
        current_context = [history[0]] + history[-10:]
    else:
        current_context = history

    # 调用模型
    res = chat_model.invoke(current_context)

    # 打印 AI 回复并存入历史
    print(f"\n✍️  AI: {res.content}")
    history.append(res)

--- 原始响应对象 ---
content='好的，好的！酷毙了！那就来个这样的标题吧：\n\n💻黑科技觉醒！AI如何秒变小说大神💥\n\n这样是不是很酷炫？记得加上一些表情符号会更吸引人哦！比如 🚀、🤖 和 📚。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 90, 'total_tokens': 147, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': '', 'id': '019c223f888d4d07a31aa1fad31623f5', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c223f-8ae1-7190-9d16-89db49a8759f-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 90, 'output_tokens': 57, 'total_tokens': 147, 'input_token_details': {}, 'output_token_details': {}}

--- 纯文本回复 ---
好的，好的！酷毙了！那就来个这样的标题吧：

💻黑科技觉醒！AI如何秒变小说大神💥

这样是不是很酷炫？记得加上一些表情符号会更吸引人哦！比如 🚀、🤖 和 📚。

--- Token 消耗详情 ---
输入: 90 | 输出: 57
AI: 你好！当然可以开始正常工作了。请问您对我的新小说有什么构想或者具体的要求吗？比如题材、情节走向或者角色设定方面，我很乐意根据您的指示进行创作。


KeyboardInterrupt: 